In [1]:
import tensorflow as tf
import numpy as np
from time import time
import os

print(tf.__version__)

2022-04-29 12:29:53.456672: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


2.3.0


### Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")
    
    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt:
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0
    
def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

### Data load & pre-processing function

In [3]:
def load_mnist():
    (train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1)  # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1)  # [N, 28, 28] -> [N, 28, 28, 1]
    
    train_data, test_data = normalize(train_data, test_data)
    
    train_labels = tf.keras.utils.to_categorical(train_labels, 10)  # [N,] -> [N, 10]
    test_labels = tf.keras.utils.to_categorical(test_labels, 10)  # [N, ] -> [N, 10]
    
    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    
    return train_data, test_data

### Performance function

In [4]:
def loss_function(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits,
                                                                  y_true=labels,
                                                                  from_logits=True))
    return loss

def accuracy_function(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_function(model, images, labels)
    return tape.gradient(loss, model.variables)

### Create model (class version)

In [5]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()
        
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Flatten())
        
        for i in range(2):
            self.model.add(tf.keras.layers.Dense(units=256,
                                                use_bias=True,
                                                kernel_initializer=weight_init))
            self.model.add(tf.keras.layers.Activation(tf.keras.activations.relu))
        
        self.model.add(tf.keras.layers.Dense(label_dim,
                                            use_bias=True,
                                            kernel_initializer=weight_init))
        
    def call(self, x, training=None, mask=None):
        x = self.model(x)
        return x

### Define data & hyper-parameter

In [6]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
            shuffle(buffer_size=100000).\
            prefetch(buffer_size=batch_size).\
            batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
            shuffle(buffer_size=100000).\
            prefetch(buffer_size=len(test_x)).\
            batch(len(test_x))

2022-04-29 12:29:55.319123: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-04-29 12:29:55.375631: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:03:00.0 name: GeForce RTX 2080 computeCapability: 7.5
coreClock: 1.71GHz coreCount: 46 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.23GiB/s
2022-04-29 12:29:55.376331: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 0000:04:00.0 name: GeForce RTX 2080 computeCapability: 7.5
coreClock: 1.71GHz coreCount: 46 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.23GiB/s
2022-04-29 12:29:55.376356: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-04-29 12:29:55.378159: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-04

### Define model & optimizer & writer

In [7]:
""" Model """
network = create_model_class(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = "checkpoints"
logs_dir = "logs"

model_dir = "nn_relu"

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

### Restore checkpoint & start train or test phase

In [8]:
if train_flag:
    checkpoint = tf.train.Checkpoint(dnn=network)
    
    # create writer for tensorborad
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()
    
    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    
    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        counter = checkpoint_counter
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))
                
                train_loss = loss_function(network, train_input, train_label)
                train_accuracy = accuracy_function(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:
                    test_accuracy = accuracy_function(network, test_input, test_label)
                
                tf.summary.scalar(name="train_loss", data=train_loss, step=counter)
                tf.summary.scalar(name="train_accuracy", data=train_accuracy, step=counter)
                tf.summary.scalar(name="test_accuracy", data=test_accuracy, step=counter)
                
                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + "-{}".format(counter))

# test phase
else:
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:
        test_accuracy = accuracy_function(network, test_input, test_label)
    
    print("test_Accuracy : %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.7084, train_loss: 2.16265225, train_accuracy: 0.4688, test_Accuracy: 0.3105
Epoch: [ 0] [    1/  468] time: 0.8579, train_loss: 2.11250591, train_accuracy: 0.3828, test_Accuracy: 0.3547
Epoch: [ 0] [    2/  468] time: 1.0079, train_loss: 2.08679152, train_accuracy: 0.3750, test_Accuracy: 0.3934
Epoch: [ 0] [    3/  468] time: 1.1513, train_loss: 1.99680877, train_accuracy: 0.4844, test_Accuracy: 0.4518
Epoch: [ 0] [    4/  468] time: 1.2905, train_loss: 1.91523623, train_accuracy: 0.4922, test_Accuracy: 0.5032
Epoch: [ 0] [    5/  468] time: 1.4341, train_loss: 1.79253864, train_accuracy: 0.6406, test_Accuracy: 0.5428
Epoch: [ 0] [    6/  468] time: 1.6030, train_loss: 1.72574234, train_accuracy: 0.6094, test_Accuracy: 0.5924
Epoch: [ 0] [    7/  468] time: 1.7537, train_loss: 1.65895975, train_accuracy: 0.6562, test_Accuracy: 0.6479
Epoch: [ 0] [    8/  468] time: 1.8994,

Epoch: [ 0] [   74/  468] time: 11.5667, train_loss: 0.42820570, train_accuracy: 0.8828, test_Accuracy: 0.9011
Epoch: [ 0] [   75/  468] time: 11.7157, train_loss: 0.46368837, train_accuracy: 0.8438, test_Accuracy: 0.9033
Epoch: [ 0] [   76/  468] time: 11.8545, train_loss: 0.43998408, train_accuracy: 0.8672, test_Accuracy: 0.9069
Epoch: [ 0] [   77/  468] time: 11.9883, train_loss: 0.42415792, train_accuracy: 0.8516, test_Accuracy: 0.9051
Epoch: [ 0] [   78/  468] time: 12.1304, train_loss: 0.47063917, train_accuracy: 0.8906, test_Accuracy: 0.9050
Epoch: [ 0] [   79/  468] time: 12.2662, train_loss: 0.31130883, train_accuracy: 0.8906, test_Accuracy: 0.9048
Epoch: [ 0] [   80/  468] time: 12.4118, train_loss: 0.25862265, train_accuracy: 0.9297, test_Accuracy: 0.9064
Epoch: [ 0] [   81/  468] time: 12.5508, train_loss: 0.39972532, train_accuracy: 0.9062, test_Accuracy: 0.9097
Epoch: [ 0] [   82/  468] time: 12.6932, train_loss: 0.27856165, train_accuracy: 0.9141, test_Accuracy: 0.9120
E

Epoch: [ 0] [  148/  468] time: 22.2479, train_loss: 0.13914813, train_accuracy: 0.9688, test_Accuracy: 0.9277
Epoch: [ 0] [  149/  468] time: 22.3864, train_loss: 0.29760540, train_accuracy: 0.9297, test_Accuracy: 0.9291
Epoch: [ 0] [  150/  468] time: 22.5200, train_loss: 0.19231647, train_accuracy: 0.9453, test_Accuracy: 0.9286
Epoch: [ 0] [  151/  468] time: 22.6618, train_loss: 0.16198337, train_accuracy: 0.9609, test_Accuracy: 0.9266
Epoch: [ 0] [  152/  468] time: 22.8042, train_loss: 0.28370148, train_accuracy: 0.9219, test_Accuracy: 0.9267
Epoch: [ 0] [  153/  468] time: 22.9440, train_loss: 0.15829584, train_accuracy: 0.9453, test_Accuracy: 0.9264
Epoch: [ 0] [  154/  468] time: 23.0892, train_loss: 0.32679251, train_accuracy: 0.9219, test_Accuracy: 0.9286
Epoch: [ 0] [  155/  468] time: 23.2310, train_loss: 0.25914246, train_accuracy: 0.9297, test_Accuracy: 0.9319
Epoch: [ 0] [  156/  468] time: 23.3635, train_loss: 0.15821329, train_accuracy: 0.9531, test_Accuracy: 0.9339
E

Epoch: [ 0] [  222/  468] time: 32.8930, train_loss: 0.13305765, train_accuracy: 0.9609, test_Accuracy: 0.9371
Epoch: [ 0] [  223/  468] time: 33.0276, train_loss: 0.16536695, train_accuracy: 0.9375, test_Accuracy: 0.9377
Epoch: [ 0] [  224/  468] time: 33.1733, train_loss: 0.13800836, train_accuracy: 0.9531, test_Accuracy: 0.9378
Epoch: [ 0] [  225/  468] time: 33.3323, train_loss: 0.18044698, train_accuracy: 0.9453, test_Accuracy: 0.9379
Epoch: [ 0] [  226/  468] time: 33.4907, train_loss: 0.21939886, train_accuracy: 0.9453, test_Accuracy: 0.9390
Epoch: [ 0] [  227/  468] time: 33.6281, train_loss: 0.29711685, train_accuracy: 0.9219, test_Accuracy: 0.9407
Epoch: [ 0] [  228/  468] time: 33.7736, train_loss: 0.20211723, train_accuracy: 0.9453, test_Accuracy: 0.9410
Epoch: [ 0] [  229/  468] time: 33.9192, train_loss: 0.20968282, train_accuracy: 0.9219, test_Accuracy: 0.9401
Epoch: [ 0] [  230/  468] time: 34.0602, train_loss: 0.17201611, train_accuracy: 0.9531, test_Accuracy: 0.9377
E

Epoch: [ 0] [  296/  468] time: 43.6259, train_loss: 0.12841758, train_accuracy: 0.9688, test_Accuracy: 0.9494
Epoch: [ 0] [  297/  468] time: 43.7606, train_loss: 0.15040430, train_accuracy: 0.9375, test_Accuracy: 0.9471
Epoch: [ 0] [  298/  468] time: 43.9055, train_loss: 0.22043446, train_accuracy: 0.9531, test_Accuracy: 0.9478
Epoch: [ 0] [  299/  468] time: 44.0477, train_loss: 0.18893392, train_accuracy: 0.9219, test_Accuracy: 0.9493
Epoch: [ 0] [  300/  468] time: 44.1923, train_loss: 0.13229105, train_accuracy: 0.9531, test_Accuracy: 0.9520
Epoch: [ 0] [  301/  468] time: 44.3207, train_loss: 0.16332811, train_accuracy: 0.9531, test_Accuracy: 0.9516
Epoch: [ 0] [  302/  468] time: 44.4618, train_loss: 0.22090212, train_accuracy: 0.9219, test_Accuracy: 0.9519
Epoch: [ 0] [  303/  468] time: 44.6088, train_loss: 0.11112845, train_accuracy: 0.9688, test_Accuracy: 0.9524
Epoch: [ 0] [  304/  468] time: 44.7450, train_loss: 0.07333806, train_accuracy: 0.9844, test_Accuracy: 0.9516
E

Epoch: [ 0] [  370/  468] time: 54.1467, train_loss: 0.20295322, train_accuracy: 0.9375, test_Accuracy: 0.9554
Epoch: [ 0] [  371/  468] time: 54.2904, train_loss: 0.26099885, train_accuracy: 0.9375, test_Accuracy: 0.9547
Epoch: [ 0] [  372/  468] time: 54.4413, train_loss: 0.13695000, train_accuracy: 0.9609, test_Accuracy: 0.9526
Epoch: [ 0] [  373/  468] time: 54.5951, train_loss: 0.12485777, train_accuracy: 0.9609, test_Accuracy: 0.9521
Epoch: [ 0] [  374/  468] time: 54.7333, train_loss: 0.11317638, train_accuracy: 0.9844, test_Accuracy: 0.9519
Epoch: [ 0] [  375/  468] time: 54.8813, train_loss: 0.18197484, train_accuracy: 0.9297, test_Accuracy: 0.9529
Epoch: [ 0] [  376/  468] time: 55.0278, train_loss: 0.08850878, train_accuracy: 0.9844, test_Accuracy: 0.9516
Epoch: [ 0] [  377/  468] time: 55.1676, train_loss: 0.12164791, train_accuracy: 0.9453, test_Accuracy: 0.9522
Epoch: [ 0] [  378/  468] time: 55.3103, train_loss: 0.16037279, train_accuracy: 0.9375, test_Accuracy: 0.9528
E

Epoch: [ 0] [  444/  468] time: 64.6088, train_loss: 0.15428077, train_accuracy: 0.9609, test_Accuracy: 0.9587
Epoch: [ 0] [  445/  468] time: 64.7449, train_loss: 0.07249561, train_accuracy: 0.9844, test_Accuracy: 0.9608
Epoch: [ 0] [  446/  468] time: 64.8867, train_loss: 0.18371147, train_accuracy: 0.9531, test_Accuracy: 0.9639
Epoch: [ 0] [  447/  468] time: 65.0199, train_loss: 0.04464229, train_accuracy: 0.9922, test_Accuracy: 0.9637
Epoch: [ 0] [  448/  468] time: 65.1588, train_loss: 0.14188732, train_accuracy: 0.9688, test_Accuracy: 0.9611
Epoch: [ 0] [  449/  468] time: 65.3027, train_loss: 0.17067927, train_accuracy: 0.9609, test_Accuracy: 0.9571
Epoch: [ 0] [  450/  468] time: 65.4483, train_loss: 0.13693339, train_accuracy: 0.9453, test_Accuracy: 0.9546
Epoch: [ 0] [  451/  468] time: 65.5895, train_loss: 0.16509859, train_accuracy: 0.9453, test_Accuracy: 0.9578
Epoch: [ 0] [  452/  468] time: 65.7365, train_loss: 0.08108756, train_accuracy: 0.9766, test_Accuracy: 0.9603
E